In [47]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time
import json
import scipy.stats as st

In [48]:
player_list = ['Michael Jordan', 'Kobe Bryant', 'LeBron James', 'Magic Johnson', 'Larry Bird', 'Bill Russell', 
               'Wilt Chamberlain', 'Kareem Abdul-Jabbar', "Shaquille O'Neal", 'Stephen Curry' ]

In [49]:
#define functions

#this function takes the 1 row career stats df from the csv and prepares it for merging
def career_cleaner(df, name):
    df = df.reset_index(drop = True)
    df = df.rename(index = {0 : name})
    df = df.transpose()
    df.index.name = "Stats"
    return df

#this function uses the player_df and takes a category and uses it to sort and assign points 
#to the players from greatest to least according to that category
def points_assigner(category):
    df = player_df.sort_values(category, ascending = False)
    points = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
    df[category + " PTS"] = points
    df = df.reset_index()
    df = df.rename(columns = {"index": "Players"})
    df = df[["Players", category + " PTS"]]
    return df

In [50]:
#import data sets and save the career stats line in each players' df
jordan_career = pd.read_csv(r"csv_files/jordan_career.csv")
jordan_career = jordan_career.iloc[[-4], ]

kobe_career = pd.read_csv(r"csv_files/kobe_career.csv")
kobe_career = kobe_career.iloc[[-1], ]

lebron_career = pd.read_csv(r"csv_files/lebron_career.csv")
lebron_career = lebron_career.iloc[[-5], ]

magic_career = pd.read_csv(r"csv_files/magic_career.csv")
magic_career = magic_career.iloc[[-1], ]

bird_career = pd.read_csv(r"csv_files/bird_career.csv")
bird_career = bird_career.iloc[[-1], ]

russell_career = pd.read_csv(r"csv_files/russell_career.csv")
russell_career = russell_career.iloc[[-1], ]

wilt_career = pd.read_csv(r"csv_files/wilt_career.csv")
wilt_career = wilt_career.iloc[[-5], ]

kareem_career = pd.read_csv(r"csv_files/kareem_career.csv")
kareem_career = kareem_career.iloc[[-4], ]

shaq_career = pd.read_csv(r"csv_files/shaq_career.csv")
shaq_career = shaq_career.iloc[[-8], ]

steph_career = pd.read_csv(r"csv_files/steph_career.csv")
steph_career = steph_career.iloc[[-1], ]

In [51]:
#clean functions for merging
jordan_career = career_cleaner(jordan_career, "Jordan")
kobe_career = career_cleaner(kobe_career, "Kobe")
lebron_career = career_cleaner(lebron_career, "Lebron")
magic_career = career_cleaner(magic_career, "Magic")
bird_career = career_cleaner(bird_career, "Bird")
russell_career = career_cleaner(russell_career, "Russell")
wilt_career = career_cleaner(wilt_career, "Wilt")
kareem_career = career_cleaner(kareem_career, "Kareem")
shaq_career = career_cleaner(shaq_career, "Shaq")
steph_career = career_cleaner(steph_career, "Steph")

In [52]:
#merge all career data together
vert_df = jordan_career.merge(kobe_career, on = "Stats")
vert_df = vert_df.merge(lebron_career, on = "Stats")
vert_df = vert_df.merge(magic_career, on = "Stats")
vert_df = vert_df.merge(bird_career, on = "Stats")
vert_df = vert_df.merge(russell_career, on = "Stats", how = "outer")
vert_df = vert_df.merge(wilt_career, on = "Stats", how = "outer")
vert_df = vert_df.merge(kareem_career, on = "Stats")
vert_df = vert_df.merge(shaq_career, on = "Stats")
vert_df = vert_df.merge(steph_career, on = "Stats")

In [53]:
#delete unnecessary statistics
vert_df = vert_df.drop(["Tm", "GS", "Lg", "MP", "Age", "Pos", "FG", "FGA", "3P", "3PA", "2P", "2PA", "2P%", "eFG%", "FT", "FTA", "ORB", "DRB", "TOV", "PF"])

In [54]:
#organize the df to have player names as indexes and stats as columns
player_df = vert_df.transpose()

In [55]:
#Assigning zeros to both Wilt and Russell for their 3 point percentages
player_df.loc["Russell", "3P%"] = 0
player_df.loc["Wilt", "3P%"] = 0

#Assigning the correct stat of blocks for both Wilt and Russell
player_df.loc["Russell", "BLK"] = 8.1
player_df.loc["Wilt", "BLK"] = 8.8

#Assigning the correct stat of steals for both Wilt and Russell
player_df.loc["Russell", "STL"] = 1.5
player_df.loc["Wilt", "STL"] = 1.5

In [65]:
#display dataframe
player_df

Stats,Season,G,FG%,3P%,FT%,TRB,AST,STL,BLK,PTS
Jordan,Career,1072,0.497,0.327,0.835,6.2,5.3,2.3,0.8,30.1
Kobe,Career,1346,0.447,0.329,0.837,5.2,4.7,1.4,0.5,25
Lebron,Career,1258,0.504,0.344,0.735,7.4,7.4,1.6,0.8,27.1
Magic,Career,906,0.52,0.303,0.848,7.2,11.2,1.9,0.4,19.5
Bird,Career,897,0.496,0.376,0.886,10,6.3,1.7,0.8,24.3
Russell,Career,963,0.44,0,0.561,22.5,4.3,1.5,8.1,15.1
Wilt,Career,1045,0.54,0,0.511,22.9,4.4,1.5,8.8,30.1
Kareem,Career,1560,0.559,0.056,0.721,11.2,3.6,0.9,2.6,24.6
Shaq,Career,1207,0.582,0.045,0.527,10.9,2.5,0.6,2.3,23.7
Steph,Career,699,0.476,0.435,0.906,4.5,6.6,1.7,0.2,23.5


In [56]:
#Creating scoreboard
scoreboard_df = pd.DataFrame({
    "Players" : player_list})

In [66]:
#Determine who has the highest field goal percentage
temp = points_assigner("FG%")
scoreboard_df = scoreboard_df.merge(temp, on = "Players", how = "right")

In [68]:
#Determine who has the highest 3-point percentage
temp = points_assigner("3P%")
scoreboard_df = scoreboard_df.merge(temp, on = "Players", how = "right")

In [69]:
#Determine who has the highest free throw percentage
temp = points_assigner("FT%")
scoreboard_df = scoreboard_df.merge(temp, on = "Players", how = "right")

In [70]:
#Determine who has the most rebounds per game
temp = points_assigner("TRB")
scoreboard_df = scoreboard_df.merge(temp, on = "Players", how = "right")

In [72]:
#Determine who has the most assists per game
temp = points_assigner("AST")
scoreboard_df = scoreboard_df.merge(temp, on = "Players", how = "right")

In [71]:
#Determine who has the most steals per game
temp = points_assigner("STL")
scoreboard_df = scoreboard_df.merge(temp, on = "Players", how = "right")

In [73]:
#Determine who has the most blocks per game
temp = points_assigner("BLK")
scoreboard_df = scoreboard_df.merge(temp, on = "Players", how = "right")

In [64]:
#Determine who has the most points per game
temp = points_assigner("PTS")
scoreboard_df = scoreboard_df.merge(temp, on = "Players", how = "right")
scoreboard_df

,Players,FG% PTS,3P% PTS,FT% PTS,TRB PTS,AST PTS,STL PTS,BLK PTS,PTS PTS
0,Shaq,10,3,2,7,1,1,7,4
1,Kareem,9,4,4,8,2,2,8,6
2,Wilt,8,1,1,10,4,4,10,9
3,Magic,7,5,8,4,10,9,2,2
4,Lebron,6,8,5,5,9,6,5,8
5,Jordan,5,6,6,3,6,10,6,10
6,Bird,4,9,9,6,7,8,4,5
7,Steph,3,10,10,1,8,7,1,3
8,Kobe,2,7,7,2,5,3,3,7
9,Russell,1,2,3,9,3,5,9,1


In [19]:
#Determine who has the highest win percentage

In [20]:
#Determine who had the most MVP awards
mvp_df = pd.read_csv(r"csv_files/mvp_history.csv")

grouped_mvps = mvp_df.groupby("Unnamed: 2").count()
grouped_mvps = grouped_mvps.reset_index()
grouped_mvps = grouped_mvps[["Unnamed: 2", "Unnamed: 0"]]

only_ours = grouped_mvps.loc[grouped_mvps["Unnamed: 2"].isin(player_list)]
only_ours

,Unnamed: 2,Unnamed: 0
1,Bill Russell,5
15,Kareem Abdul-Jabbar,6
19,Kobe Bryant,1
20,Larry Bird,3
21,LeBron James,4
22,Magic Johnson,3
23,Michael Jordan,5
28,Shaquille O'Neal,1
29,Stephen Curry,2
34,Wilt Chamberlain,4


In [21]:
#Determine who had the most championship wins

In [22]:
#Determine who won rookie of the year

In [23]:
#Determine who won the most defensive player of the year awards

In [24]:
#Determine who has the most all-star appearances

In [25]:
#Determine who has the most playoff appearances

In [26]:
#Map of markers of location of where each players played